In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

# Extracting Information from Text Data Assignment

In [43]:
import spacy
import string
import textacy
import itertools
import pandas as pd
from nltk import pos_tag
from rake_nltk import Rake
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import tree2conlltags
from gensim.summarization import keywords
from nltk.chunk.regexp import RegexpParser
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from spacy import displacy

<IPython.core.display.Javascript object>

In [3]:
# !pip install textacy --quiet
# !pip install rake_nltk --quiet

<IPython.core.display.Javascript object>

### Read the CNN Lite plain text file articles into a corpus using the NLTK's PlaintextCorpusReader.

In [5]:
DOC_PATTERN = r".*\.txt"
corpus = PlaintextCorpusReader("corpata/cnn_lite/", DOC_PATTERN)

<IPython.core.display.Javascript object>

### Iterate through the fileids in the corpus, extract the raw text of each document, and store them in a list.

In [8]:
docs = []
for fileid in corpus.fileids():
    document = corpus.raw(fileid)
    docs.append(document)

<IPython.core.display.Javascript object>

### Extract the top 5 keywords from every document in the corpus. Print them and compare the differences in keywords among the documents.

In [9]:
doc = docs[0]

<IPython.core.display.Javascript object>

In [14]:
# keywords(doc, words=5, lemmatize=True).split("\n")
key_words = keywords(doc, words=5, split=True, lemmatize=True)

['sanders', 'campaign', 'news', 'democrats', 'recent']

<IPython.core.display.Javascript object>

In [31]:
for doc in docs:
    top_5 = keywords(doc, words=5, split=True, lemmatize=True)
    print(top_5)
    print("------")

['sanders', 'campaign', 'news', 'democrats', 'recent']
------
['courts', 'trump', 'states', 'general', 'judges']
------
['politically', 'news', 'people', 'america', 'trump']
------
['erdogan', 'said', 'trump', 'meet', 'turkish']
------
['laurent', 'said', 'news', 'student', 'age']
------
['schools', 'high', 'town', 'gun', 'clarita']
------
['politically', 'people', 'american', 'guns', 'news']
------
['trump', 'november', 'border', 'facts', 'said']
------
['politically', 'presidents', 'likely', 'community', 'american']
------
['visitors', 'park', 'national', 'african', 'giraffes']
------
['livingstone', 'christmas', 'clarke', 'said', 'wham']
------
['trump', 'taylor', 'president', 'news', 'republican']
------
['bevin', 'general', 'kentucky', 'thursday', 'cnn']
------
['gaynor', 'hurricane', 'carper', 'changes', 'said']
------
['trump', 'democratic', 'republicans', 'presidency', 'news']
------
['rehaag', 'room', 'media', 'woman', 'later']
------
['snacks', 'hostess', 'twinkies', 'cereals

<IPython.core.display.Javascript object>

### Extract the top 3 keyphrases from each document, print them, and compare the differences.

In [18]:
r = Rake()
r.extract_keywords_from_text(doc)
key_phrases = r.get_ranked_phrases_with_scores()
key_phrases[:3]

[(64.35555555555555,
  'win ," campaign manager faiz shakir told cnn last month'),
 (52.0, 'billionaire former new york city mayor michael bloomberg'),
 (33.0, 'former vice president joe biden came')]

<IPython.core.display.Javascript object>

In [34]:
r = Rake()
for doc in docs:
    r.extract_keywords_from_text(doc)
    top_3_phrases = r.get_ranked_phrases_with_scores()[0:3]
    print(top_3_phrases)
    print("------")

[(64.35555555555555, 'win ," campaign manager faiz shakir told cnn last month'), (52.0, 'billionaire former new york city mayor michael bloomberg'), (33.0, 'former vice president joe biden came')]
------
[(43.25, 'attorneys general offices could gather financial information'), (25.25, 'attorney general brian frosh called'), (25.0, '© 2019 cable news network')]
------
[(26.015151515151516, 'america ," cnn opinion invited readers'), (23.0, '© 2019 cable news network'), (23.0, 'seen increasingly vast gaps develop')]
------
[(29.42820512820513, 'senior administration official said trump first raised'), (29.2, 'turkish president recep tayyip erdoğan pulled'), (25.0, '© 2019 cable news network')]
------
[(25.0, '© 2019 cable news network'), (20.625, 'unusual ," said sjoerd hulshof'), (14.8, 'discover new things ."')]
------
[(82.4, 'make quick judgments ," saugus high student lèan aguilar said'), (25.0, '© 2019 cable news network'), (24.0, 'santa clarita valley signal reported')]
------
[(29

<IPython.core.display.Javascript object>

### Identify and extract the named entities in each document, filtering out the numeric types. Print them and compare the differences between documents.

In [36]:
nlp = spacy.load("en_core_web_sm")
spacy_doc = nlp(doc)

entities = [[entity.text, entity.label_] for entity in spacy_doc.ents]

<IPython.core.display.Javascript object>

In [55]:
numeric_types = ["DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL"]
nlp = spacy.load("en_core_web_sm")
for doc in docs:
    spacy_doc = nlp(doc)

    entities = [
        [entity.text, entity.label_]
        for entity in spacy_doc.ents
        if entity.label_ not in numeric_types
    ]
    names = pd.Series([ent[0] for ent in entities])
    labels = pd.Series([ent[1] for ent in entities])
    print("Top Named Entities:")
    print(names.value_counts()[:3])
    print("Top Entity Label:")
    print(labels.value_counts()[:3])
    #     print(entities)
    print("-----------------------------------")

Top Named Entities:
Sanders    23
CNN         6
Iowa        6
dtype: int64
Top Entity Label:
ORG       37
GPE       29
PERSON    20
dtype: int64
-----------------------------------
Top Named Entities:
Trump       6
Maryland    3
CNN         3
dtype: int64
Top Entity Label:
ORG       19
GPE       11
PERSON     5
dtype: int64
-----------------------------------
Top Named Entities:
America     18
Trump       16
Democrat     4
dtype: int64
Top Entity Label:
GPE       62
PERSON    32
ORG       30
dtype: int64
-----------------------------------
Top Named Entities:
Erdoğan    12
Turkey     11
S-400       6
dtype: int64
Top Entity Label:
GPE       22
PERSON    22
ORG       19
dtype: int64
-----------------------------------
Top Named Entities:
Laurent    11
CNN         5
Lydia       3
dtype: int64
Top Entity Label:
ORG       17
PERSON     8
GPE        5
dtype: int64
-----------------------------------
Top Named Entities:
CNN                                     4
Saugus High School            

Top Named Entities:
Taylor        13
Republican     9
Trump          7
dtype: int64
Top Entity Label:
PERSON    48
NORP      30
ORG       27
dtype: int64
-----------------------------------
Top Named Entities:
CNN         5
Facebook    3
KPIX        3
dtype: int64
Top Entity Label:
ORG       22
PERSON     5
GPE        4
dtype: int64
-----------------------------------
Top Named Entities:
Patrick          11
CNN               5
Massachusetts     3
dtype: int64
Top Entity Label:
GPE       18
PERSON    16
ORG       15
dtype: int64
-----------------------------------
Top Named Entities:
CNN        3
US         2
Midwest    1
dtype: int64
Top Entity Label:
ORG    9
GPE    4
LOC    2
dtype: int64
-----------------------------------
Top Named Entities:
CNN        3
Jameson    2
Willow     2
dtype: int64
Top Entity Label:
PERSON         8
ORG            8
WORK_OF_ART    4
dtype: int64
-----------------------------------
Top Named Entities:
Ukraine     21
Zelensky    18
Trump       14
dtype: in

<IPython.core.display.Javascript object>

In [27]:
displacy.serve(spacy_doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


<IPython.core.display.Javascript object>

### For every document in the corpus, iterate over every sentence, extract any SVO triples, print them, and compare.

In [29]:
results = []

for sent in spacy_doc.sents:
    svo = textacy.extract.subject_verb_object_triples(sent)
    results += svo

results

[(Cortez, capped, debut),
 (we, allow, to happen),
 (We, don't watch, race),
 (Sanders, has charted, revival),
 (him, gaining, steam),
 (that, welcomed, him),
 (Sanders, joked, following),
 (I, 've got, arteries),
 (supporters, are saying, much),
 (staff, are saying, much),
 (backing, bolstered, argument),
 (campaign, attracting, coalition),
 (CNN poll, showed, Sanders),
 (one, touched, digits),
 (surveys, show, group),
 (campaign, puts, it),
 (person, seems, different),
 (that, has vaulted, him),
 (Sanders, entered, primary),
 (he, challenged, Clinton),
 (Sanders, began, bid),
 (he, expand, base),
 (Iowa, caucuses kick, primary),
 (Biden, offers, contrast),
 (South Bend, has emerged, foe),
 (Mayor Pete Buttigieg, has emerged, foe),
 (he, can grow, it),
 (Sanders, is taking, source),
 (I, know, that),
 (Sanders, invited, television cameras),
 (Sanders, invited, reporters),
 (he, 's taken, tone),
 (I, thank, you),
 (I, want, to say),
 (Sanders, delivered, punchline),
 (Sanders, injected

<IPython.core.display.Javascript object>

In [54]:
for doc in docs:
    spacy_doc = nlp(doc)
    results = []

    for sent in spacy_doc.sents:
        svo = textacy.extract.subject_verb_object_triples(sent)
        results += svo

    print(results)

[(Cortez, capped, debut), (we, allow, to happen), (We, don't watch, race), (Sanders, has charted, revival), (him, gaining, steam), (that, welcomed, him), (Sanders, joked, following), (I, 've got, arteries), (supporters, are saying, much), (staff, are saying, much), (backing, bolstered, argument), (campaign, attracting, coalition), (CNN poll, showed, Sanders), (one, touched, digits), (surveys, show, group), (campaign, puts, it), (person, seems, different), (that, has vaulted, him), (Sanders, entered, primary), (he, challenged, Clinton), (Sanders, began, bid), (he, expand, base), (Iowa, caucuses kick, primary), (Biden, offers, contrast), (South Bend, has emerged, foe), (Mayor Pete Buttigieg, has emerged, foe), (he, can grow, it), (Sanders, is taking, source), (I, know, that), (Sanders, invited, television cameras), (Sanders, invited, reporters), (he, 's taken, tone), (I, thank, you), (I, want, to say), (Sanders, delivered, punchline), (Sanders, injected, some), (we, tell, Trump), (we, te

[(President Donald Trump, made, claims), (Trump, made, claims), (it, has proven, accurate), (Washington Post, fabricated, sources), (Trump, tried, get), (he, had committed, crimes), (Post, invented, sources), (news outlets, followed, Post scoop), (Trump, has made, claims), (government, approved, facility), (Trump, visited, facility), (I, got, them), (I, said, Let), (Trump, gave, speech), (Trump, recited, one), (We, check, polls), (he, tweets, claim), (he, keeps, saying), (he, takes, step), (They, 're having, people), (witnesses, have had, knowledge), (they, misconstrued, it), (they, shouldn't be having, hearings), (they, shouldn't be having, hearings), (It, makes, sense), (me, tell, you), (I, know, gentleman), (he, says, that), (I, said, that), (he, hasn't changed, testimony), (Sondland, had not changed, testimony), (You, can read, fact check), (that, made, conversation), (he, didn't use, word), (Trump, criticize, Schiff), (Schiff, didn't use, word), (You, can read, breakdown), (Presid

[(Diplomat Bill Taylor, played, role), (Taylor, joined, Kent), (Taylor, brought, life), (Taylor, brought, conviction), (which, led, him), (which, led, to accuse), (Taylor, expanded, testimony), (who, overheard, Trump), (Ukrainians, would undertake, investigations), (he, heard, them), (Taylor, didn't try, to embellish), (He, refused, to speculate), (Taylor, managed, to share), (Republicans, shrug, impeachment hearings), (House phone call, doesn't show, pressure), (House staffers, are refusing, to cooperate), (this, becomes, story), (Jordan, 're going, have), (we, 're going, have), (all, going, to say), (we, got, to impeach), (I, shrank, it), (that, explain, accusations), (Ukraine, raises, concerns), (Sondland, delivers, ultimatum), (Taylor, raises, concerns), (Inspector general, informs, Congress), (aide, had overheard, Trump), (he, doesn't recall, it), (I, never heard, this), (House, votes, to impeach), (she, suggests, solution), (senators, use, ballot), (senators, could vote, minds), 

[(Hong Kong, looked, set), (Hong Kong, looked, set), (level, has reached, heights), (state media, warning, protesters), (police, confronted, protesters), (who, have taken, footbridge), (protesters, built, barricades), (protesters, barricaded, themselves), (Police, fired, canisters), (students, have been asked, to leave), (they, had evacuated, students), (MTR, continued, to suspend), (schools, have suspended, classes), (protests, have taken, turn), (he, began, clearing), (protesters, hurling, bricks), (government supporters, hurling, bricks), (what, is believed, be), (escalation, raised, questions), (run television channel, issued, editorial), (We, have had, talking), (We, have had, persuasion), (We, have had, warnings), (country, will never accept, situation), (It, echoed, editorial), (We, warn, protesters), (who, are coerced, be), (you, can make, call), (protesters, to cross, it), (you, are going, have), (protesters, seek, to breach), (Adam Ni, told, CNN), (government, unveiled, range

[(Washington, is declining, to open), (Office, is declining, to open), (impeachment inquiry, taking, place), (they, had sent, letter), (they, had sent, to), (who, had requested, review), (Democrats, expressed, disappointment), (Department Office, has declined, to investigate), (it, would appear, to have hindered), (office, pursue, review), (Trump administration, had placed, hold), (President, attempted, to use), (White House, has offered, accounts), (what, triggered, Trump), (what, triggered, change), (what, triggered, release), (President, released, aid), (House officials, been made, aware), (Administration officials, have tried, to pin), (They, have pointed, to ensure), (process, took, place), (Laura Cooper, told, lawmakers), (Volker, had told, her), (he, was attempting, to lift), (Ukrainians, deliver, statement), (they, would launch, investigations), (it, is conducting, review), (Trump administration, violated, appropriations law), (Government Accountability Office, initiated, revie

[(she, will be taking, step), (Jameson, 's going, to start), (star, praised, husband), (she, will celebrate, years), (he, follows, me)]
[(President Donald Trump, used, US tax dollars), (President Donald Trump, used, to pressure), (that, could damage, Biden), (who, played, President), (Zelensky, wins, landslide victory), (Ukraine, needs, US), (Ukraine, needs, aid), (Zelensky, seeks, meeting), (York Times, publishes, report), (which, had placed, him), (Ukrainians, to investigate, arrangement), (it, represents, time), (members, were requiring, quo), (meeting, featured, Department officials), (meeting, featured, officials), (suggestion, alarmed, Bolton), (Ambassador, characterized, request), (Congress, had directed, to be sent), (Trump, can congratulate, Zelensky), (Trump, brings, investigations), (which, inhibited, interaction), (call, triggers, concerns), (Trump, asks, Sondland), (They, discuss, topics), (which, does not mention, corruption investigations), (Volker, advise, leadership), 

<IPython.core.display.Javascript object>